# DEAP EEG Data Exploration

This notebook explores the DEAP dataset and demonstrates the preprocessing pipeline.

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import custom modules
from preprocessing import load_bdf_to_trials, preprocess_trials
from utils.viz import plot_eeg_waveform

%matplotlib inline
sns.set_style('whitegrid')

## 1. Load Raw Data

In [ ]:
# Load one subject's data
subject_id = 1
bdf_path = f'../data/raw/s{subject_id:02d}.bdf'

print(f"Loading data from {bdf_path}...")
trials = load_bdf_to_trials(bdf_path, n_channels=48)

print(f"Data shape: {trials.shape}")
print(f"Number of trials: {trials.shape[0]}")
print(f"Number of channels: {trials.shape[1]}")
print(f"Samples per trial: {trials.shape[2]}")
print(f"Duration per trial: {trials.shape[2] / 512:.1f} seconds @ 512 Hz")

## 2. Visualize Raw EEG

In [ ]:
# Plot first trial, first 8 channels
trial_idx = 0
trial_data = trials[trial_idx]

plot_eeg_waveform(
    trial_data,
    fs=512,
    channels_to_plot=list(range(8)),
    title=f'Subject {subject_id} - Trial {trial_idx} - Raw EEG'
)

## 3. Data Statistics

In [ ]:
# Compute statistics
print("Raw data statistics:")
print(f"Mean: {trials.mean():.4f}")
print(f"Std: {trials.std():.4f}")
print(f"Min: {trials.min():.4f}")
print(f"Max: {trials.max():.4f}")

# Distribution
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(trials[0, 0, :], bins=50, alpha=0.7)
plt.xlabel('Amplitude')
plt.ylabel('Frequency')
plt.title('Distribution of Channel 0 Values')

plt.subplot(1, 2, 2)
channel_means = trials[0].mean(axis=1)
plt.bar(range(len(channel_means)), channel_means)
plt.xlabel('Channel')
plt.ylabel('Mean Amplitude')
plt.title('Mean Amplitude per Channel')
plt.tight_layout()
plt.show()

## 4. Preprocessing

In [ ]:
# Preprocess trials
print("Preprocessing trials...")
processed = preprocess_trials(
    trials[:5],  # Process first 5 trials for speed
    fs_in=512,
    fs_out=128,
    do_ica=False  # Skip ICA for speed in demo
)

print(f"Processed shape: {processed.shape}")
print(f"Downsampled from {trials.shape[2]} to {processed.shape[2]} samples")

## 5. Compare Raw vs Preprocessed

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Raw
time_raw = np.arange(trials[0, 0, :].shape[0]) / 512
axes[0].plot(time_raw[:2000], trials[0, 0, :2000], linewidth=0.5)
axes[0].set_title('Raw EEG (Channel 0, first 4 seconds)')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True, alpha=0.3)

# Preprocessed
time_proc = np.arange(processed[0, 0, :].shape[0]) / 128
axes[1].plot(time_proc[:500], processed[0, 0, :500], linewidth=0.5, color='orange')
axes[1].set_title('Preprocessed EEG (Channel 0, first 4 seconds)')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Normalized Amplitude')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nPreprocessed statistics:")
print(f"Mean: {processed.mean():.4f} (should be ~0)")
print(f"Std: {processed.std():.4f} (should be ~1)")

## 6. Frequency Analysis

In [ ]:
from scipy import signal

# Compute power spectral density
fs = 128
f, psd = signal.welch(processed[0, 0, :], fs=fs, nperseg=256)

plt.figure(figsize=(12, 4))
plt.semilogy(f, psd)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density')
plt.title('PSD of Preprocessed EEG (Channel 0)')
plt.xlim([0, 50])
plt.grid(True, alpha=0.3)

# Mark frequency bands
bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
    'Gamma': (30, 50)
}

for band_name, (low, high) in bands.items():
    plt.axvspan(low, high, alpha=0.1, label=band_name)

plt.legend()
plt.show()

## 7. Windowing for Model Input

In [ ]:
# Create sliding windows
window_size = 256  # 2 seconds @ 128 Hz
step_size = 128    # 1 second @ 128 Hz

trial = processed[0]
n_samples = trial.shape[1]

windows = []
for start in range(0, n_samples - window_size + 1, step_size):
    window = trial[:, start:start + window_size]
    windows.append(window)

windows = np.array(windows)
print(f"Created {len(windows)} windows of shape {windows[0].shape}")
print(f"Total windows from {processed.shape[0]} trials: {len(windows) * processed.shape[0]}")

## 8. Inter-subject Variability

In [ ]:
# Load multiple subjects
subject_means = []
subject_stds = []

for sid in range(1, 6):  # First 5 subjects
    try:
        bdf_path = f'../data/raw/s{sid:02d}.bdf'
        trials_s = load_bdf_to_trials(bdf_path, n_channels=48)
        subject_means.append(trials_s.mean())
        subject_stds.append(trials_s.std())
    except:
        print(f"Could not load subject {sid}")

if subject_means:
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.bar(range(1, len(subject_means) + 1), subject_means)
    plt.xlabel('Subject ID')
    plt.ylabel('Mean Amplitude')
    plt.title('Mean Amplitude per Subject')
    
    plt.subplot(1, 2, 2)
    plt.bar(range(1, len(subject_stds) + 1), subject_stds)
    plt.xlabel('Subject ID')
    plt.ylabel('Std Amplitude')
    plt.title('Std Amplitude per Subject')
    
    plt.tight_layout()
    plt.show()

## Summary

This notebook demonstrated:
1. Loading DEAP .bdf files
2. Visualizing raw EEG signals
3. Preprocessing pipeline (filtering, downsampling, normalization)
4. Frequency analysis
5. Creating sliding windows for model input
6. Inter-subject variability

Next steps:
- Run full preprocessing: `python src/preprocessing.py`
- Train model: `python src/train.py`
- Evaluate: `python src/eval.py`